# Exercise 1: Create Phecodes and Demographics

This notebook will extract ICD9CM and ICD10CM codes from the CDR. It will then transform those codes into phecodes for analysis. It also creates a basic demographics table.

I used a Standard VM with 16 CPUs and 104GB RAM.

## Install PheWAS Package

First, we will install the PheWAS package. The kernel may require a restart after this. Fortunately, this install should stay with our persistent disk.

In [ ]:
if(!require(PheWAS)) devtools::install_github("PheWAS/PheWAS", upgrade=FALSE)
library(PheWAS)
library(tidyverse)

## Setup bigrquery

This is a convenience function we can use to query the default CDR for this workspace.

In [ ]:
library(bigrquery)  # BigQuery R client.

## BigQuery setup.
BILLING_PROJECT_ID <- Sys.getenv('GOOGLE_PROJECT')
# Get the BigQuery curated dataset for the current workspace context.
CDR <- Sys.getenv('WORKSPACE_CDR')
# Bucket
WORKSPACE_BUCKET <- Sys.getenv('WORKSPACE_BUCKET')

#Query
bq <- function(query) {bq_table_download(bq_project_query(
    BILLING_PROJECT_ID, page_size = 25000,
    query=query, default_dataset = CDR ))
}

## Setup Python bigquery

We will additionally use the python bigquery library with the reticulate package. It supports querying GCS files as BQ tables, which we can leverage to map phecodes in BQ instead of locally.

In [ ]:
library(reticulate)
bigquery=import("google.cloud.bigquery")

client = bigquery$Client()

### Save phecode map

We first export the phecode map from the PheWAS Package and save it to our workspace bucket.

In [ ]:
# Create the phecode map
expanded_phecode = inner_join(PheWAS::phecode_map, rename(PheWAS::phecode_rollup_map, phecode=code)) %>% transmute(vocabulary_id, concept_code=code, phecode=phecode_unrolled)
write_csv(expanded_phecode, file="expanded_phecode.csv")
system2("gsutil",args = c("cp","expanded_phecode.csv",WORKSPACE_BUCKET))

### Configure the phecode map table for BQ

We are adding our GCS phecode table as an external table for BQ. This provides the metadata BQ needs to interpret the file.

In [ ]:
phecode = bigquery$ExternalConfig('CSV')
phecode$source_uris = c(sprintf("%s/expanded_phecode.csv",WORKSPACE_BUCKET))
phecode$schema = c(
    bigquery$SchemaField('vocabulary_id', 'STRING'),
    bigquery$SchemaField('concept_code', 'STRING'),
    bigquery$SchemaField('phecode', 'STRING')
)
py_set_attr(phecode$options,"skip_leading_rows", "1")  # optionally skip header row

## Extract phecodes

We use BQ to select all ICD9CM and ICD10CM codes, map them to phecodes, and aggregate them by selecting unique counts of dates per phecode. The BQ configuration code includes the above definition of our external table.

In [ ]:
icds=sprintf("with all_codes as (select * from (
select distinct person_id, vocabulary_id, concept_code, condition_start_date as date
from condition_occurrence join concept on (condition_source_concept_id=concept_id)
where vocabulary_id in ('ICD9CM','ICD10CM')
union distinct
select distinct person_id, vocabulary_id, concept_code, observation_date as date
from observation join concept on (observation_source_concept_id=concept_id)
where vocabulary_id in ('ICD9CM','ICD10CM')
union distinct
select distinct person_id, vocabulary_id, concept_code, procedure_date as date
from procedure_occurrence join concept on (procedure_source_concept_id=concept_id)
where vocabulary_id in ('ICD9CM','ICD10CM')
union distinct
select distinct person_id, vocabulary_id, concept_code, measurement_date as date
from measurement join concept on (measurement_source_concept_id=concept_id)
where vocabulary_id in ('ICD9CM','ICD10CM')
))
select person_id, phecode, count(distinct date) as count from
all_codes join expanded_phecode using (vocabulary_id, concept_code)
group by person_id, phecode
")

In [ ]:
job_config = bigquery$QueryJobConfig()
job_config$table_definitions = c("expanded_phecode" = phecode)
job_config$default_dataset = CDR
query_job = client$query(icds, job_config=job_config)  # API request

data = query_job$to_dataframe()

In [ ]:
dim(data)

### Extract total population list

It is possible that some healthy controls may not have any phecodes. We will find all individuals with any ICD9CM or ICD10CM billing codes to consider them the denominator. This should give us a reasonably consistent overall EHR population- true healthy controls should at least have well visit codes.

In [ ]:
# Get individuals with EHR data (for control list)
all_code_query="with all_codes as (select * from (
select distinct person_id, src_id
from condition_occurrence join condition_occurrence_ext using (condition_occurrence_id) join concept on (condition_source_concept_id=concept_id)
where vocabulary_id in ('ICD9CM','ICD10CM') and src_id like 'EHR%'
union distinct
select distinct person_id, src_id
from observation join observation_ext using (observation_id) join concept on (observation_source_concept_id=concept_id)
where vocabulary_id in ('ICD9CM','ICD10CM') and src_id like 'EHR%'
union distinct
select distinct person_id, src_id
from procedure_occurrence join procedure_occurrence_ext using (procedure_occurrence_id) join concept on (procedure_source_concept_id=concept_id)
where vocabulary_id in ('ICD9CM','ICD10CM') and src_id like 'EHR%'
union distinct
select distinct person_id, src_id
from measurement join measurement_ext using (measurement_id) join concept on (measurement_source_concept_id=concept_id)
where vocabulary_id in ('ICD9CM','ICD10CM') and src_id like 'EHR%'
))
select distinct person_id, src_id from
all_codes
"

In [ ]:
job_config = bigquery$QueryJobConfig()
job_config$default_dataset = CDR
query_job = client$query(all_code_query, job_config=job_config)  # API request

ehr_inds = query_job$to_dataframe()

### Multiple EHR sites

As a note, some individuals have data from multiple EHR sites, which is expected.

In [ ]:
ehr_inds %>% group_by(person_id) %>% summarize(n_site=n()) %>% group_by(n_site) %>% summarize(n())

### Create distinct participant ID list

In [ ]:
ehr_person_ids= (ehr_inds %>% select(person_id) %>% distinct())[[1]]

### Create sex covariates

We next create a covariate of self reported sex at birth to exclude individuals from sex-specific phecodes.

In [ ]:
job_config = bigquery$QueryJobConfig()
job_config$default_dataset = CDR
query_job = client$query("select person_id, 
case when sex_at_birth_concept_id=45878463 then 'F'
when sex_at_birth_concept_id=45880669 then 'M'
else '0'
end as sex
from person", 
                         job_config=job_config)  # API request

sex_info = query_job$to_dataframe()

In [ ]:
table(sex_info$sex)

## Create Phenotypes

Using the PheWAS Package, we will create our phecode table. This incorporates the already mapped phecodes, a minimum code count of 2, our sex exclusion list, and an overall population list.

In [ ]:
phe_table = createPhenotypes(data %>% transmute(person_id, vocabulary_id="phecode",phecode, count), 
                             translate=FALSE, min.code.count=2, add.phecode.exclusions=FALSE,
                             id.sex=sex_info,full.population.ids=ehr_person_ids)

In [ ]:
dim(phe_table)

### Save the file

Next, we save the file. We can also upload the file to our workspace bucket so we can retrieve it later. Note that this file is quite large, so will take some time to upload.

In [ ]:
write_csv(phe_table,file="phecode_table.csv")

In [ ]:
# Optionally upload the file to your bucket for safe-keeping
#system2("gsutil",args = c("cp","phecode_table.csv",WORKSPACE_BUCKET))

## Create Demographics

We will also create a small demographics table for use. This is based on the OMOP person table, which contains self-reported information.

In [ ]:
job_config = bigquery$QueryJobConfig()
job_config$default_dataset = CDR
query_job = client$query("select * from person", 
                         job_config=job_config)  # API request

demo_raw = query_job$to_dataframe()

In [ ]:
#View the raw data if it's of interest!
#head(demo_raw)

In [ ]:
demo_raw %>% group_by(ethnicity_source_value) %>% summarize(n())

### Simplify the data for covariates

Note that we will not use all of these covariates in our analysis- you may find them useful or not for your work.

In [ ]:
demo=demo_raw %>% transmute(person_id, 
                            age=2023-year_of_birth, 
                            race_w_nh=((race_source_value=="WhatRaceEthnicity_White")&(ethnicity_source_value=="Not Hispanic")),
                            sex_ab_F=sex_at_birth_source_value=="SexAtBirth_Female",
                           race_source_value, ethnicity_source_value, sex_at_birth_source_value, year_of_birth)

In [ ]:
#View the data as is
#head(demo)

In [ ]:
#View scrambled data
demo %>% transmute(across(-person_id, \(x) sample(x))) %>% head()

In [ ]:
demo %>% group_by(race_w_nh) %>% summarize(n())

### Save the file

Write it to the local VM then save to the workspace bucket.

In [ ]:
write_csv(demo,file="demo.csv")

In [ ]:
system2("gsutil",args = c("cp","demo.csv",WORKSPACE_BUCKET))